# Notebook description

#### What is this notebook for? 
This this the first notebook of a serie of five. Its main purpose is to collect the data that will be used for the rest of the project. 


The process of data collection will be divided in multiple processes: 
1. Create a list of pages/account where I found bots' comments 
2. For each, I look at their last 50+ posts to collect the ID of the post 
3. Once done, loop through those thousands of IDs to collect the comments
4. 



collected the IDs of  from multiple pages suseptible to have bots commenting on their posts. I selected those pages manually by making sure they're all bots' targets. Then, I'll loop through each post thanks to it's ID. On instagram, each post url is made as `instagram.com/p/{postid}`

## Script

## Import modules and UDFs

In [2]:
#Personnal module, functions I use often
import src.webscraping as mw
import src.useful as mu


# Better print 
from tqdm import tqdm

# To become Dr Strange 
import time 
from datetime import datetime

# Basic data 
import pandas as pd
import numpy as np

# Store data 
import sqlite3
import json
from flatten_json import flatten

# Move things around locally
import shutil

# Fetch instagram data 
from instaloader import Instaloader, Profile
from instaloader.exceptions import ProfileNotExistsException
import urllib
from splinter import Browser

import os

### Setting notebook preferences

In [12]:
pd.set_option("display.max_columns", None)

In [3]:
# Creating SQL database to store all the data for this project
database = "data/main_database.sqlite"
con = sqlite3.connect(database)

## 1. Collect post ids

List a number of instagram accounts that have bots commenting on their posts. Here I'm looping through a page list that is targeted by bots and collect the posts_ids one by one.

In [14]:
# During the first collection time I had 5 more
pages = ["nfl", "championsleague", "mercedesamgf1", "ESPN", "bleacherreport", "houseofhighlights", "nba", "worldstar", "grmdaily", "pubity", 
         "meme.ig", "brgridiron", "lakers", "ballislife", "nflonfox", "nflnetwork", "espnnfl", "cbssports", "thecheckdown"]

In [8]:
df_post_ids = pd.DataFrame(columns=["post_id", "page"])
post_per_page = 50

# Lunch browser
browser = Browser('chrome')

for page in pages:
    browser.visit(f"https://www.instagram.com/{page}/")
    postids = []
    for i in list(range(10))[::-1]:
        print(i, end="\r")
        time.sleep(1)

    while len(postids) < post_per_page:
        # Scroll up and then down each time helps the page to not bug
        browser.execute_script("window.scrollTo(0, 0);")
        time.sleep(0.5)
        browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)

        # Change the browser to a beautiful soup object where I can get the posts id
        soup = mw.bsoup(browser)

        # Instagram doesn't show in the html the posts it of the post we don't see so I need to slowly scroll down to collect each of them.
        browser.execute_script("window.scrollBy(0, 200);")

        for element in soup.find_all("a"):
            link = element.get("href")
            
            # We can find the posts id by looking into a tags that have an attribute of href.
            if "/p/" in link:
                postids.append(link.replace("/p/", "").replace("/", ""))

        # Removing duplicates
        postids = list(set(postids))

        time.sleep(np.random.randint(30)/10)


        post_page_id = [[post_id, page] for post_id in postids]

        # Create a df with post_ids and save it in the db
        df_new_post_ids = pd.DataFrame(post_page_id, columns=["post_id", "page_id"])
        df_new_post_ids.to_sql("post_ids", con, if_exists="append", index=False)
        time.sleep(5)


## 2. Scrape comments

In [22]:
# Query post_id not in comments table 
query = """
SELECT  
    DISTINCT
    post_id
    , page_name
FROM post_ids 
"""

# Loadind post_ids from db
posts_ids_to_scrape = pd.read_sql_query(query, con)
posts_ids_to_scrape

,post_id,page_name
0,CY23Dx-hQzz,br_hoops
1,CYhycdhh__L,br_hoops
2,CXpjIIagKAx,br_hoops
3,CYxi2doBjWu,br_hoops
4,CYDVbgjhJEq,br_hoops
...,...,...
7826,CWIYygLtmsJ,mercedesamgf1
7827,CZhKJh_uJne,mercedesamgf1
7828,CWROwfUMZR4,mercedesamgf1
7829,CYWw8V4tRPD,mercedesamgf1


In [415]:
# Scraping and storing comments from post_ids:
np.random.shuffle(posts_ids_to_scrape)
for post_id in tqdm(posts_ids_to_scrape):
    browser.visit(f"https://www.instagram.com/p/{post_id}/")
    
    while True: 
        # Get page soup 
        soup = mw.bsoup(browser)
        if soup.find("h2", class_="_a9zc") != None: 
            break

    # Get days since posted
    post_posted_time = soup.find("time", class_="_a9ze _a9zf").get("datetime")
    now = datetime.now()
    days_diff = (now - pd.to_datetime(post_posted_time[:-1])).days

    # I only keep what was posted less than a month ago
    if days_diff > 31: 
        continue
    
    df_post_comments = pd.DataFrame(columns=["post_id", "page", "legend", "post_posted_time", "username", 
                                                       "full_comment_data", "comment", "comment_posted_time", 
                                                       "time_since_posted", "comments_likes", "replies",
                                                       "time_now"])
    for comment_block in soup.find_all("ul", class_="_a9ym"):
        page = soup.find("h2", class_="_a9zc").text 
        legend = soup.find("div", class_="_a9zs").text 
        time_since_posted = comment_block.find("time", class_="_a9ze _a9zf").text 
        username = comment_block.find("h3", class_="_a9zc").text 
        comment_posted_time = comment_block.find("time", class_="_a9ze _a9zf").get("datetime")
        comments_likes = comment_block.find("button", class_="_a9ze").text 
        comment = comment_block.find("div", class_="_a9zs").text  
        replies = comment_block.find("li", class_="_a9yg").text if comment_block.find("li", class_="_a9yg") != None else ""
        full_comment_data = comment_block.text
        time_now = datetime.now()

        # Add comment values to dataframe
        df_post_comments.loc[len(df_post_comments)] = [post_id, page, legend, post_posted_time, username, 
                                                       full_comment_data, comment, comment_posted_time, 
                                                       time_since_posted, comments_likes, replies,
                                                       time_now]

        df_post_comments["period"] = "summer"

        
    df_post_comments.to_sql("comments", con, if_exists="append", index=False)
    time.sleep(np.random.randint(40, 60))

## 3. Collect user data from comments

In [176]:
# Instanciate Instaloader
L = Instaloader()

def fetch_user_data (username):
    '''Function to fetch an Instagram user's public data.

    Parameter: 
        username str: username of the user to collect the 
        data from.
    '''

    try:
        profile = Profile.from_username(L.context, username)
    except ProfileNotExistsException:
        return f"{username} does not exists anymore"
        
    data = profile.__dict__
    del data["_context"]
    json_object = json.dumps(data, indent = 2)   
    with open(f"data/users_json/{username}_user_profile_data.json", 'w') as file_object:  
        json.dump(json_object, file_object) 

In [174]:
def convert_json (username):
    # Open json 
    try:
        json_file = open(f"data/users_json/{username}_user_profile_data.json")
        data = json.loads(json.load(json_file))["_node"]
    except json.decoder.JSONDecodeError:
        return


    # useles json keys
    ban = ['edge_felix_video_timeline', 'edge_owner_to_timeline_media', 'edge_saved_media', 
            'edge_media_collections', 'edge_related_profiles']

    # Defining basic keys. Those are name, follower count, bio, etc. Basic infos
    basic_keys = [key for key in data.keys() if key not in ban]

    basic_info = flatten({key:data[key] for key in basic_keys})
    df_current_user = pd.DataFrame([basic_info])
    # username = df_current_user.loc[0, "username"]

    # Getting the data for all posts that are contained in lists. 
    df_current_user["video_count"] = data['edge_felix_video_timeline']["count"]
    df_current_user["post_count"] = data['edge_owner_to_timeline_media']["count"]

    last_12_posts = dict()
    posts = data['edge_owner_to_timeline_media']["edges"]
    last_12_posts["username"] = data["username"]
    last_12_posts["video_views"] = [post["node"]["video_view_count"] if "video_view_count" in post["node"].keys() else np.nan for post in posts]
    last_12_posts["display_url"] = [post["node"]["display_url"] for post in posts]
    last_12_posts["thumbnail_src"] = [post["node"]["thumbnail_src"] for post in posts]
    last_12_posts["accessibility_caption"] = [post["node"]["accessibility_caption"] for post in posts]
    last_12_posts["is_video"] = [post["node"]["is_video"] for post in posts]
    last_12_posts["likes"] = [post["node"]["edge_liked_by"]["count"] for post in posts]
    last_12_posts["comments"] = [post["node"]["edge_media_to_comment"]["count"] for post in posts]
    last_12_posts["timestamp"] = [post["node"]["taken_at_timestamp"] for post in posts]
    df_last_12_posts = pd.DataFrame(last_12_posts)


    # Changing list type to str as sqlite3 doesn't accept this type
    list_features = ['bio_links', 'biography_with_entities_entities', 'edge_mutual_followed_by_edges', 'pronouns']
    for column in list_features: 
        if column in df_current_user.columns:
            df_current_user[column] = df_current_user[column].apply(lambda x:  "_LIST_SEPARATOR_".join(x))

    return df_current_user, df_last_12_posts

In [187]:
all_folders = ['/Volumes/Lacie /igbotdata/website_screenshot',
'/Users/marclamy/Desktop/code/all ig bot folders/igbot_final/data/website_screenshot',
'/Users/marclamy/Desktop/code/all ig bot folders/instagram_bot_detection/data/url_screenshot']

all_files_full_path = [mu.list_files(folder) for folder in tqdm(all_folders)]

100%|██████████| 3/3 [00:06<00:00,  2.08s/it]


In [188]:
all_files_full_path = [file for lst in all_files_full_path for file in lst]
dict_all_files_full_path = {fp.split('/')[-1]: fp for fp in all_files_full_path}


clean_dict_all_files_full_path = {}

for key, value in dict_all_files_full_path.items():
    if key not in clean_dict_all_files_full_path.keys():
        clean_dict_all_files_full_path[key] = value

len(clean_dict_all_files_full_path)

17537

In [191]:
done_users_photos = mu.list_files('data/photos/user_profile_pictures', common=False)
done_12_photos = mu.list_files('data/photos/user_last_12_posts', common=False)

len(done_users_photos) + len(done_12_photos)

449762

In [192]:
keys = list(clean_dict_all_files_full_path.keys())
np.random.shuffle(keys)
clean_dict_all_files_full_path = {key: clean_dict_all_files_full_path[key] for key in keys}
len(clean_dict_all_files_full_path)

17537

In [197]:
# clean_dict_all_files_full_path

In [1]:
# done_users_photos = mu.list_files('data/photos/user_profile_pictures')
# done_users_photos = [file.split('/')[-1] for file in done_users_photos]
# done_12_photos = mu.list_files('data/photos/user_last_12_posts')
# done_12_photos = [file.split('/')[-1] for file in done_12_photos]

all_df = []
for file_name, current_path in tqdm(list(clean_dict_all_files_full_path.items())):
    df_photos = pd.DataFrame()
    new_path = 'data/photos/'
    # print(f'{file_name}')


    if file_name not in done_users_photos and file_name not in done_12_photos:
        # if '_pp_user_photo' in file_name:
        username = file_name.replace('_website_photo.png', '')
        new_path = new_path + 'bio_url_screenshot/' + file_name
        shutil.copy(current_path, new_path)
        df_photos.loc[len(df_photos), ['username', 'photo_type', 'path']] = [username, 'bio_url', new_path]

        # else:
        #     photo_index = file_name.split('_')[-3]
        #     username = file_name.replace(f'_{photo_index}_user_photo.png', '')
        #     new_path = new_path + 'user_last_12_posts/' + file_name
        #     shutil.copy(current_path, new_path)
        #     df_photos.loc[len(df_photos), ['username', 'photo_type', 'path']] = [username, f'{photo_index}', new_path]

    all_df.append(df_photos)

df = pd.concat(all_df, axis=0)

NameError: name 'tqdm' is not defined

In [200]:
df.to_sql('photos', con, if_exists='append', index=False)

17537

In [184]:
# Query post_id not in comments table 
query = """
SELECT  
    DISTINCT
    username
FROM comments 
WHERE 1=1
    --AND username NOT IN (SELECT username FROM users)
    --AND username NOT IN (SELECT username FROM errors where error_type='user_not_found')"""

# Loadind usernames to scrape from db
usernames_to_scrape = pd.read_sql_query(query, con)["username"]
print(f"Usernames to scrape: {len(usernames_to_scrape)}")


for index, username in enumerate(tqdm(usernames_to_scrape[:5])): 
    # print(username, username, username, username)
    fetch_user_data(username)
    
    try:
        df_current_user, df_current_last_12_posts = convert_json(username)
    except FileNotFoundError: # If an error happenned while fetching the data, no file
        continue

    # Not all users have the same number of columns returned and SQL needs same cols so got to use this way
    if index != 0: # No table exists before first index
        df_users = pd.read_sql_query('select * from users', con)
    df_users = pd.concat([df_users, df_current_user]).drop_duplicates()
    df_users.to_sql("users", con, if_exists="replace", index=False)
    df_current_last_12_posts.to_sql("last_12_posts", con, if_exists="append", index=False)


    # Profile pic
    for username, profile_pic_url in df_current_user[["username",  "profile_pic_url"]].values: 
        try:
            urllib.request.urlretrieve(profile_pic_url, f"data/photos/test/{username}_pp_user_photo.png")
        except Exception as e:
            print(username, e, end='\r')

    # Last 12 posts
    df_current_last_12_posts = df_current_last_12_posts.reset_index()
    for username, display_url, index in df_current_last_12_posts[["username", "display_url", "index"]].values: 
        urllib.request.urlretrieve(display_url, f"data/photos/test/{username}_{index}_user_photo.png") 



### 3.4 Screenshot bio url landing page

In [186]:
# Query users having a link and remove those with NA
query = """
SELECT 
    DISTINCT 
    username
    , external_url 
FROM users 
WHERE external_url IS NOT NULL"""

df_user_urls = pd.read_sql_query(query, con)
df_user_urls

,username,external_url
0,mr.145th,https://onlyfans.com/letmeholdthatsoul
1,bmw122787,https://bit.ly/3qrZahY
2,stepan_ryabenko,https://teleg.run/stepan_ryabenko
3,supersonicadu,http://www.youtube.com/user/TheSonic2124
4,lauren_viglietta,https://vs.co/7f9fa25d
...,...,...
19160,joshlozano_,http://wer1hoops.com/
19161,qurmaca,https://t.me/qurmaca
19162,crossedcourt,http://crossedcourt.com/
19163,lessy.morgan63,https://elastic-joliot-e02363.netlify.app/


In [ ]:
browser = Browser("chrome")
for username, external_url in tqdm(df_user_urls.values):
    try:
        browser.visit(external_url)
        browser.driver.save_screenshot(f"data/url_screenshot/{username}_external_url_screenshot.png")
    except:
        ...

100%|██████████| 36/36 [02:28<00:00,  4.12s/it]


# PLayground

In [ ]:
path = "/Users/marclamy/Desktop/code/igbot_final/data/2_comments"

In [ ]:
# Make up all_comments.csv
all_files = mu.list_files(path)
df_all_comments_csv = pd.concat([pd.read_csv(file) for file in tqdm(all_files)], axis=0)

100%|██████████| 6094/6094 [00:06<00:00, 938.55it/s] 


In [ ]:
df_all_comments_csv = df_all_comments_csv.rename({"postid": "post_id", 
                                                  "data_collected_time": "time_now"
                                                  , "comment_comments_count": "replies"}, axis=1).drop("post_likes", axis=1)
df_all_comments_csv["period"] = "winter"
df_all_comments_csv["time_since_posted"] = pd.to_datetime(df_all_comments_csv["comment_posted_time"]) - pd.to_datetime(df_all_comments_csv["post_posted_time"])

df_all_comments_csv.dropna().head().columns

Index(['page', 'post_id', 'legend', 'post_posted_time', 'username',
       'full_comment_data', 'comment', 'comment_posted_time', 'comments_likes',
       'replies', 'time_now', 'period', 'time_since_posted'],
      dtype='object')

In [ ]:
list(zip(sorted(df_comments.columns), sorted(df_all_comments_csv.columns)))

[('comment', 'comment'),
 ('comment_posted_time', 'comment_posted_time'),
 ('comments_likes', 'comments_likes'),
 ('full_comment_data', 'full_comment_data'),
 ('legend', 'legend'),
 ('page', 'page'),
 ('period', 'period'),
 ('post_id', 'post_id'),
 ('post_posted_time', 'post_posted_time'),
 ('replies', 'replies'),
 ('time_now', 'time_now'),
 ('time_since_posted', 'time_since_posted'),
 ('username', 'username')]

In [ ]:
df_comments.to_sql("comments", con)

1746

In [ ]:
df_comments = pd.read_sql_query("select * from comments", con)
cols = df_comments.columns
df_all_comments_csv[cols]

df_concat = pd.concat([df_comments, df_all_comments_csv], axis=0)

DatabaseError: Execution failed on sql 'select * from comments': no such table: comments

In [ ]:
# df_concat["comment_posted_time"] = df_concat["comment_posted_time"]
df_concat["comment_posted_time"] = pd.to_datetime(df_concat["comment_posted_time"]).astype(int).astype(str)

In [ ]:
df_concat.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 136211 entries, 0 to 11
Data columns (total 13 columns):
 #   Column               Non-Null Count   Dtype 
---  ------               --------------   ----- 
 0   post_id              136211 non-null  object
 1   page                 136211 non-null  object
 2   legend               136211 non-null  object
 3   post_posted_time     136211 non-null  object
 4   username             136211 non-null  object
 5   full_comment_data    136211 non-null  object
 6   comment              136211 non-null  object
 7   comment_posted_time  136211 non-null  object
 8   time_since_posted    136211 non-null  object
 9   comments_likes       136211 non-null  object
 10  replies              136211 non-null  object
 11  time_now             136211 non-null  object
 12  period               136211 non-null  object
dtypes: object(13)
memory usage: 14.5+ MB


In [ ]:
# df_concat.to_sql("comments", con, if_exists="replace", index=False)